# Prerequisites

- Python 3.10.4

> Warning: Installation from conda environment may take few minutes

Configuring conda environment

```cmd
conda activate ca2_env
conda install matplotlib
conda install pandas
conda install nltk
conda install scikit-learn
conda install jsonpickle
conda install -c conda-forge textblob
conda install -c conda-forge tweepy
conda install -c conda-forge pycountry
conda install -c conda-forge wordcloud
conda install -c conda-forge langdetect
```

References:
https://towardsdatascience.com/step-by-step-twitter-sentiment-analysis-in-python-d6f650ade58d




In [3]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jsonpickle
import os
import nltk
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#nltk.download('brown')
#nltk.download('vader_lexicon')

In [2]:
# Authentication
api_key = "Y0hENmxMwkILuxxY0yb9NHG0L"
api_secret = "33OSFPPFNWlN3S9CvL5ok4ClBLjUdviC4zzpLs9aRBwJVzHPMA"
bearerToken = "AAAAAAAAAAAAAAAAAAAAABoHcAEAAAAA78rV2Phg2vUbwTdcJ2QPGGji65M%3DsTr8DETODttIuwkHy9czi663YUpUPLDOVY2VqUzmBdshiJmLT3"

auth = tweepy.OAuth2BearerHandler(bearerToken)
api = tweepy.API(auth)

In [180]:
#Download tweets to json file
def percentage(part,whole):
     return 100 * float(part)/float(whole)

search_words = "(agriculture OR farmers) (price OR cost) (place_country:IE) (lang:en)"
environment = "development"
fromDate = "202201010000"
tweets = tweepy.Cursor(
            api.search_full_archive,
            label=environment,
            query=search_words,
            fromDate=fromDate).items()

In [ ]:
# Save tweets
pd.json_normalize([t._json for t in tweets]).to_csv("../data/twitter/agriculture_farmers_2022.csv")

In [131]:
# Read from csv
tweets_pd = pd.read_csv("../data/twitter/agriculture_farmers_2022.csv", index_col=0)

In [132]:
tweets_pd.columns[tweets_pd.columns.str.endswith("text")]

Index(['text', 'extended_tweet.full_text', 'quoted_status.text',
       'quoted_status.extended_tweet.full_text'],
      dtype='object')

In [133]:
for c in tweets_pd.columns:
    print(c)

created_at
id
id_str
text
display_text_range
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
contributors
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
favorited
retweeted
filter_level
lang
matching_rules
user.id
user.id_str
user.name
user.screen_name
user.location
user.url
user.description
user.translator_type
user.protected
user.verified
user.followers_count
user.friends_count
user.listed_count
user.favourites_count
user.statuses_count
user.created_at
user.utc_offset
user.time_zone
user.geo_enabled
user.lang
user.contributors_enabled
user.is_translator
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_background_tile
user.profile_link_color
user.profile_sidebar_border_color
user.profile_sidebar_fill_color
user.profile_text_color
user.profile_use_background_image
user.profile_image_url
user.profil

In [134]:
tweets_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Columns: 170 entries, created_at to quoted_status_permalink.display
dtypes: bool(13), float64(44), int64(14), object(99)
memory usage: 6.2+ KB


In [135]:
# Replace truncated with extended text
tweets_pd["text"] = tweets_pd.apply(lambda r: r["extended_tweet.full_text"] if r["truncated"] else r["text"], axis=1)
tweets_pd["quoted_status.text"] = tweets_pd.apply(lambda r: r["quoted_status.extended_tweet.full_text"] if r["quoted_status.truncated"] else r["quoted_status.text"], axis=1)


> split dataset by columns. Get quoted status info

In [144]:
# Status tweets
tweets_status_pd = tweets_pd[[c for c in tweets_pd.columns if not(c.startswith("quoted"))]]
tweets_status_pd.rename({"place.full_name":"place"}, axis=1, inplace=True)


C:\Users\rmsry\AppData\Local\Temp\ipykernel_15264\903030612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_status_pd.rename({"place.full_name":"place"}, axis=1, inplace=True)


In [155]:
# Quote status tweets
tweets_quoted_status_pd = tweets_pd[tweets_pd["is_quote_status"]][tweets_pd.columns[tweets_pd.columns.str.startswith("quoted")]]
# rename columns
tweets_quoted_status_pd.rename(mapper=lambda c: c.replace("quoted_status.",""), axis='columns', inplace=True)
# add dummy "place.bounding_box.type" and "place.bounding_box.coordinates"
tweets_quoted_status_pd["place.bounding_box.type"] = None
tweets_quoted_status_pd["place.bounding_box.coordinates"] = None

In [174]:
import numpy as np
columns = ["id","geo","coordinates","reply_count","favorite_count", "place", "text" ,"user.location", "place.bounding_box.type", "place.bounding_box.coordinates"]
arr1 = tweets_quoted_status_pd[columns].to_numpy()
arr2 = tweets_status_pd[columns].to_numpy()
tweets_final_pd = pd.DataFrame(np.concatenate((arr1,arr2), axis=0), columns = columns)


In [176]:
tweets_final_pd.to_csv("final.csv")

In [50]:
#Sentiment Analysis
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
 
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1
    print(tweet)

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

NameError: name 'tweets' is not defined

In [40]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  10
positive number:  3
negative number:  2
neutral number:  5
